In [1]:
import pandas as pd
import csv
import random

In [2]:
csv.field_size_limit(10**6*10)

131072

In [3]:
# load a df from cleaned_normalized_df_no_grouping.pkl
df = pd.read_pickle('cleaned_normalized_df_no_grouping.pkl')

In [5]:
df

,FictionId,ChapterId,content,Order
0,24,365929.0,"<p>Lorem ipsum dolor sit amet, consectetur adi...",1.0
1,24,371170.0,<div>\n<p><strong>Lorem Ipsum</strong> is simp...,3.0
2,24,385417.0,<p>Go away. Let me test stuff in peace!</p>\n<...,2.0
3,24,485769.0,"<table style=""border-collapse: collapse; width...",4.0
4,24,485990.0,<p>Nullam quis ante. Donec vitae orci sed dolo...,5.0
...,...,...,...,...
1243341,9997,112254.0,"<p>Have you ever wonder, what would an apocaly...",1.0
1243342,9997,113818.0,"<p>In a space full of nothingness, where there...",2.0
1243343,9997,115147.0,"<p>Calm, almost tranquil atmosphere. That is V...",3.0
1243344,9998,112255.0,<p>“It has been a long time since one of your ...,1.0


In [6]:
len(df)

1243346

In [7]:
chapters = len(df)

# .85/s per chapter
seconds_to_upload = .85*chapters
minutes_to_upload = seconds_to_upload/60
hours_to_upload = minutes_to_upload/60
days_to_upload = hours_to_upload/24

days_to_upload

12.231991898148147

In [4]:
chunk_size = len(df) // 24
chunks = [df.iloc[i:i + chunk_size] for i in range(0, len(df), chunk_size)]

print(len(chunks))

# check that the cumulative length of the chunks is the same as the length of the df
sum([len(chunk) for chunk in chunks]) == len(df)

25


True

In [ ]:
# save each chunk to disk as pickle
for i, chunk in enumerate(chunks):
    chunk.to_pickle(f'chunk_{i}.pkl')

In [26]:
num_rows = len(df)

all_found = True
# Randomly select 5 row indices
random_indice = random.sample(range(0, num_rows), 1)
random_range = []
for i in range(0, 5):
    random_range.append(random_indice[0] + i)

# Sort the DataFrame based on the selected indices
sampled_rows = df.iloc[random_range]


for _, df_row in sampled_rows.iterrows():
    found = False
    for chunk in chunks:
        for _, chunk_row in chunk.iterrows():
            if df_row['FictionId'] == chunk_row['FictionId'] and df_row['ChapterId'] == chunk_row['ChapterId']:
                found = True
                break
        if found:
            break
    
    if not found:
        all_found = False
        break

all_found


True

: 

In [ ]:
import json
import requests
import os
from dotenv import load_dotenv

load_dotenv()

api_key = os.environ.get('API_KEY')
api_url = os.environ.get('API_URL')
url = api_url + '/index_document'

class IndexDocumentRequest:
    def __init__(self, doc_html, story_id, index):
        self.doc_html = doc_html
        self.story_id = story_id
        self.index = index

    def get_payload(self):
        json_payload = {
            'doc_html': self.doc_html,
            'story_id': self.story_id,
            'index': self.index
        }
        return json.dumps(json_payload)
    
    def send_post_request(self):
        stringified_json_payload = self.get_payload()
        headers = {"Content-Type": "application/json", "Authorization":api_key}
        req_result = requests.post(url, data=stringified_json_payload, headers=headers)

        if req_result.status_code != 200:
            req_error = req_result.text
            print(req_error)

In [12]:
def create_random_chunks():
    num_rows = df.shape[0]

    # Randomly select 5 row indices
    random_indice = random.sample(range(0, num_rows), 1)
    random_range = []
    for i in range(0, 5):
        random_range.append(random_indice[0] + i)

    # Sort the DataFrame based on the selected indices
    sampled_rows = df.iloc[random_range]

    # get the first 10 rows with a FictionId of 24
    rows_24 = df.loc[df['FictionId'] == 24].head(10)

    for index, row in rows_24.iterrows():
        doc_html = row['content']
        story_id = int(row['FictionId'])
        index = int(row['Order'])
        index_doc_request = IndexDocumentRequest(doc_html, story_id, index)
        index_doc_request.send_post_request()

In [13]:
for i in range(0, 10):
    create_random_chunks()

{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}
{"message":"Empty Document","error_code":"0026"}


In [9]:
from qdrant_client import QdrantClient

client = QdrantClient(url="http://localhost:6333", api_key="qdrant_pass")

client.count(collection_name="doc_group_5")

CountResult(count=2)